In [ ]:
!pip install transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/AcMa

/content/drive/MyDrive/AcMa


In [4]:
#!git clone https://github.com/Davido111200/QuestionAnswering_demoVbdi.git

In [11]:
%cd /content/drive/MyDrive/AcMa/QuestionAnswering_demoVbdi/mrc/PhoQA

/content/drive/MyDrive/AcMa/QuestionAnswering_demoVbdi/mrc/PhoQA


In [15]:
!python main.py --resume

Streaming output truncated to the last 5000 lines.
 96% 1704/1779 [21:42<00:58,  1.29it/s]epoch: 0, step: 1704, loss: 1.4543628692626953
 96% 1705/1779 [21:43<00:57,  1.29it/s]epoch: 0, step: 1705, loss: 1.1775567531585693
 96% 1706/1779 [21:43<00:56,  1.29it/s]epoch: 0, step: 1706, loss: 1.2303329706192017
 96% 1707/1779 [21:44<00:55,  1.29it/s]epoch: 0, step: 1707, loss: 1.3248568773269653
 96% 1708/1779 [21:45<00:55,  1.29it/s]epoch: 0, step: 1708, loss: 1.3944472074508667
 96% 1709/1779 [21:46<00:54,  1.29it/s]epoch: 0, step: 1709, loss: 0.8478567004203796
 96% 1710/1779 [21:47<00:53,  1.29it/s]epoch: 0, step: 1710, loss: 1.2322582006454468
 96% 1711/1779 [21:47<00:52,  1.29it/s]epoch: 0, step: 1711, loss: 1.1978144645690918
 96% 1712/1779 [21:48<00:51,  1.29it/s]epoch: 0, step: 1712, loss: 1.5444810390472412
 96% 1713/1779 [21:49<00:51,  1.29it/s]epoch: 0, step: 1713, loss: 1.0688352584838867
 96% 1714/1779 [21:50<00:50,  1.29it/s]epoch: 0, step: 1714, loss: 1.5177671909332275
 96

In [21]:
ls

app.py  LICENSE  ngrok.py            QuestionAnswering_Notebook.ipynb
data/   mrc/     passage_retrieval/  README.md


In [22]:
!git add .

^C


In [23]:
!git commit -m 'huycq-da code duoc mot chut'


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@7a1aadd035b0.(none)')


In [24]:
!git push origin main

fatal: could not read Username for 'https://github.com': No such device or address


In [8]:
import sys
import os
import json
import codecs
from tqdm import tqdm
import py_vncorenlp
import re

def preprocess(text, start_span):
  count = 0
  new_str = ''

  # Block below is to remove extra white spaces BEFORE start_span
  removed_extra_spaces = " ".join(text[:start_span].split())
  n_extra_spaces = len(text[:start_span]) - len(removed_extra_spaces)
  
  count -= n_extra_spaces

  removed_all_extra_spaces = " ".join(text.split())
  # add extra spaces where needed
  for idx, character in enumerate(removed_all_extra_spaces):
    if idx == 0 or idx == len(removed_all_extra_spaces)-1:
      new_str += removed_all_extra_spaces[idx]
    else:
      if character.isalnum():
        new_str += character
      else:
        if character == ' ':
          new_str += character
        else:
          if removed_all_extra_spaces[idx-1]==' ' and removed_all_extra_spaces[idx+1]==' ':
            new_str += character
          elif removed_all_extra_spaces[idx-1]==' ' and removed_all_extra_spaces[idx+1]!=' ':
            new_str += character
            new_str += ' '
            if idx<start_span:
              count +=1
          elif removed_all_extra_spaces[idx-1]!=' ' and removed_all_extra_spaces[idx+1]==' ':
            new_str += ' '
            new_str += character
            if idx<start_span:
              count+=1
          elif removed_all_extra_spaces[idx-1]!=' ' and removed_all_extra_spaces[idx+1]!=' ':
            new_str += ' '
            new_str += character
            new_str += ' '
            if idx<start_span:
              count+=2
  new_start_span = start_span + count
  return new_str, new_start_span

class PhoBertTokenizer:
    def __init__(self):
        self.word_segmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content/drive/MyDrive/AcMa/QuestionAnswering_demoVbdi/mrc/PhoQA/dataset/vncorenlp')
        #.word_segment(text)

    def tokenize(self, text):
        text = self.word_segmenter.word_segment(text)
        text = ' '.join(text)
        return text
        """result.
        for sentence in sentences:
            tokens.extend(self.word_segmenter.tokenize(sentence))"""

    """def tokenize(self, text):
        text = self.word_segmenter.word_segment(text)
        text = ' '.join(text)
        return text
        return ' '.join(self.word_segmenter.tokenize(text)[0])#.replace("( ", "(").replace(" )", ")").replace(' ...', '...').replace(' ,', ',').replace("( ", "(").replace(" ?", "?")"""


def word_segment(file_path, word_segmenter):
    with open(file_path, 'r') as f:
        data = json.load(f)

    for i, d in enumerate(tqdm(data['data'])):
        for j, p in enumerate(d['paragraphs']):
            context = p['context']
            
            for k, q in enumerate(p['qas']):
                data['data'][i]['paragraphs'][j]['qas'][k]['question'] = word_segmenter.tokenize(q['question'])

                if 'answers' in q:
                    for l, a in enumerate(q['answers']):
                        data['data'][i]['paragraphs'][j]['qas'][k]['answers'][l]['text'] = word_segmenter.tokenize(a['text'])
                        _ ,start_postion = preprocess(context, a['answer_start'])
                        data['data'][i]['paragraphs'][j]['qas'][k]['answers'][l]['answer_start'] = start_postion
                if 'plausible_answers' in q:
                    for l, a in enumerate(q['plausible_answers']):
                        data['data'][i]['paragraphs'][j]['qas'][k]['plausible_answers'][l]['text'] = word_segmenter.tokenize(a['text'])
                        _ ,start_postion = preprocess(context, a['answer_start'])
                        data['data'][i]['paragraphs'][j]['qas'][k]['plausible_answers'][l]['answer_start'] = start_postion

            #print(len(data['data'][i]['paragraphs'][j]['context']))
            context ,_ = preprocess(context, a['answer_start'])
            data['data'][i]['paragraphs'][j]['context'] = word_segmenter.tokenize(context)

            #print(len(data['data'][i]['paragraphs'][j]['context']))
            
            """for k, c in enumerate(p['context']):
                print(c)
                data['data'][i]['paragraphs'][j]['context'][k] = word_segmenter.tokenize(c)"""

    #with open("train_word_seg.json", 'w', encoding='uft-8') as f:
    #    json.dump(data, f, indent=4, ensure_ascii=False)

    with codecs.open("/content/drive/MyDrive/AcMa/QuestionAnswering_demoVbdi/mrc/PhoQA/dataset/w_seg/train.json", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)



if __name__ == '__main__':

    word_segmenter = PhoBertTokenizer()
    word_segment('/content/drive/MyDrive/AcMa/QuestionAnswering_demoVbdi/mrc/PhoQA/dataset/train.json', word_segmenter)
    #a = "Những đối tác xuất khẩu chính của Nhật là Hoa Kỳ 22.9%, Trung Quốc 13.4%, Hàn Quốc 7.8%, Đài Loan 7.3% và Hồng Kông 6.1% (2005). Những mặt hàng xuất khẩu chính của Nhật là thiết bị giao thông vận tải, xe cơ giới, hàng điện tử, máy móc điện tử và hóa chất. Do hạn chế về tài nguyên thiên nhiên để duy trì sự phát triển của nền kinh tế, Nhật Bản phải phụ thuộc vào các quốc gia khác về phần nguyên liệu vì vậy đất nước này nhập khẩu rất nhiều loại hàng hóa đa dạng. Đối tác nhập khẩu chính của Nhật là Trung Quốc 21%, Hoa Kỳ 12.7%, A Rập Xê Út 5.5%, UAE 4.9%, Australia 4.7%, Hàn Quốc 4.7% và Indonesia 4% (số liệu 2005). Những mặt hàng nhập khẩu chính của Nhật Bản là máy móc, thiết bị, chất đốt, thực phẩm (đặc biệt là thịt bò), hóa chất, nguyên liệu dệt may và những nguyên liệu cho các ngành công nghiệp của đất nước. Nhìn chung, Đối tác buôn bán tổng thể lớn nhất của Nhật Bản là Trung Quốc."


100%|██████████| 138/138 [00:50<00:00,  2.74it/s]


In [9]:
pip install py_vncorenlp --target=='/content/drive/MyDrive/AcMa/QuestionAnswering_demoVbdi/mrc/PhoQA/dataset/vncorenlp' --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached py_vncorenlp-0.1.3-py3-none-any.whl
  Using cached pyjnius-1.4.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)


In [9]:
word_segment('/content/drive/MyDrive/AcMa/QuestionAnswering_demoVbdi/mrc/PhoQA/dataset/test.json', word_segmenter)

100%|██████████| 19/19 [00:10<00:00,  1.83it/s]
